# Connect to Athena, Query required data and Store it in Data Frame

In [4]:
#import boto3, pandas, StringIO libraries
import boto3
import pandas as pd
from io import StringIO

In [5]:
#credentials to access to AWS resources
AWS_ACCESS_KEY ="AKIAVBZTVDGMMETDS"
AWS_SECRET_KEY ="UtmKI+gY9Fx9CmL9/os5+y8AxAQmZk55dA/"
AWS_REGION ="us-east-2"
SCHEMA_NAME ='covid_db'
S3_STAGING_DIR ='s3://output-bucket/output/'
S3_BUCKET_NAME ='output-bucket'
S3_OUTPUT_DIRECTORY='output'

In [6]:
#Build Athena client using boto3
athena_client = boto3.client(
    'athena',
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)

In [7]:
Dict={}
#function to load query results and download from s3. It will return DataFrame
def download_load_query(client: boto3.client,query_response: Dict) -> pd.DataFrame:
    while True:
        try:
            # This function only loads the first 1000 rows
            client.get_query_results(
                QueryExecutionId=query_response["QueryExecutionId"]
            )
            break
        except Exception as err:
            if "not yet finished" in str(err):
                pass
            else:
                raise err
    
    temp_file_location: str = "athena_query_results.csv"
    #create an S3 client using boto3
    s3_client = boto3.client(
        's3',
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION
    )
    s3_client.list_buckets()
    #download the query result file directly from S3 and store it in local workspace
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,
    )
    return pd.read_csv(temp_file_location)

In [8]:
#Use Athena Client to run queries on enigma_jhud table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': 'ad985011-6f80-43fd-8ef3-156791de7727',
 'ResponseMetadata': {'RequestId': 'd8845e73-5ab0-4a66-b89d-32b5644d1648',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:12 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd8845e73-5ab0-4a66-b89d-32b5644d1648'},
  'RetryAttempts': 0}}

In [9]:
#call the function using athena_client, response of query as parameters and store the resulted dataframe
enigma_jhud=download_load_query(athena_client,response)

In [10]:
#enigma_jhud dataframe first 5 rows
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [11]:
#Use Athena Client to run queries on nytimes_data_in_usa_countyus_county table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_countyus_county",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '770ede05-af00-4d14-932c-eddb58d0541b',
 'ResponseMetadata': {'RequestId': '51c12ba5-a8ef-47dc-92bc-01d803fbaa7a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:22 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '51c12ba5-a8ef-47dc-92bc-01d803fbaa7a'},
  'RetryAttempts': 0}}

In [12]:
#call the function using athena_client, response of query as parameters
nytimes_data_in_usa_countyus_county=download_load_query(athena_client,response)

#nytimes_data_in_usa_countyus_county first 5 rows
nytimes_data_in_usa_countyus_county.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1.0,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1.0,0.0
3,2020-01-24,Cook,Illinois,17031.0,1.0,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1.0,0.0


In [13]:
#Use Athena Client to run queries on nytimes_data_in_usa_statesus_states table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM nytimes_data_in_usa_statesus_states",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': 'd65dca1c-831d-4223-9ce7-b4b54a454a3f',
 'ResponseMetadata': {'RequestId': 'b7780479-8a24-45e2-a3f0-e75c63cb788c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:24 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b7780479-8a24-45e2-a3f0-e75c63cb788c'},
  'RetryAttempts': 0}}

In [14]:
#call the function using athena_client, response of query as parameters
nytimes_data_in_usa_statesus_states=download_load_query(athena_client,response)

#nytimes_data_in_usa_statesus_states first 5 rows
nytimes_data_in_usa_statesus_states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [15]:
#Use Athena Client to run queries on rearc_covid_19_testing_data_states_dailystates_daily table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_states_dailystates_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '961802ad-9796-41fb-9fc3-e94c2a71cda8',
 'ResponseMetadata': {'RequestId': '1134e9cc-d872-41a4-bdae-b0af5283e51f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:25 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1134e9cc-d872-41a4-bdae-b0af5283e51f'},
  'RetryAttempts': 0}}

In [16]:
#call the function using athena_client, response of query as parameters
rearc_covid_19_testing_data_states_dailystates_daily=download_load_query(athena_client,response)

#rearc_covid_19_testing_data_states_dailystates_daily first 5 rows
rearc_covid_19_testing_data_states_dailystates_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0.0,0.0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0.0,0.0,0.0,0.0,0.0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1.0,0.0,997207b430824ea40b8eb8506c19a93e07bc972e,0.0,0.0,0.0,0.0,0.0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22.0,11.0,50921aeefba3e30d31623aa495b47fb2ecc72fae,0.0,0.0,0.0,0.0,0.0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0.0,0.0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0.0,0.0,0.0,0.0,0.0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5.0,44.0,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0.0,0.0,0.0,0.0,0.0,NaN


In [17]:
#Use Athena Client to run queries on rearc_covid_19_testing_data_us_dailyus_daily table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_dailyus_daily",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '8aa14cd4-23f2-4670-9476-619c36044ffc',
 'ResponseMetadata': {'RequestId': 'e7aa92f5-7de2-49e2-afeb-3ece5dae635a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:28 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e7aa92f5-7de2-49e2-afeb-3ece5dae635a'},
  'RetryAttempts': 0}}

In [18]:
#call the function using athena_client, response of query as parameters
rearc_covid_19_testing_data_us_dailyus_daily=download_load_query(athena_client,response)

#rearc_covid_19_testing_data_us_dailyus_daily first 5 rows
rearc_covid_19_testing_data_us_dailyus_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [19]:
#Use Athena Client to run queries on rearc_covid_19_testing_data_us_total_latestus_total_latest table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_covid_19_testing_data_us_total_latestus_total_latest",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '85ca1082-2386-4f49-a1cc-97b72ac9f513',
 'ResponseMetadata': {'RequestId': '9e5bc5fb-6198-4083-968d-882b24d44c55',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:30 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '9e5bc5fb-6198-4083-968d-882b24d44c55'},
  'RetryAttempts': 0}}

In [20]:
#call the function using athena_client, response of query as parameters
rearc_covid_19_testing_data_us_total_latestus_total_latest=download_load_query(athena_client,response)

#rearc_covid_19_testing_data_us_total_latestus_total_latest first 5 rows
rearc_covid_19_testing_data_us_total_latestus_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [21]:
#Use Athena Client to run queries on rearc_usa_hospital_bedsrearc_usa_hospital_beds table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM rearc_usa_hospital_beds",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '67c71b6c-5aa6-43bf-8726-cdcce5b8b08a',
 'ResponseMetadata': {'RequestId': 'ed0fc958-fa5e-4087-8319-f93df479c794',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:31 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ed0fc958-fa5e-4087-8319-f93df479c794'},
  'RetryAttempts': 0}}

In [22]:
#call the function using athena_client, response of query as parameters
rearc_usa_hospital_beds=download_load_query(athena_client,response)

#rearc_usa_hospital_beds first 5 rows
rearc_usa_hospital_beds.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
0,1,Phoenix VA Health Care System (AKA Carl T Hayd...,VA Hospital,650 E Indian School Rd,NaN,Phoenix,AZ,85012,Maricopa,Arizona,...,129.0,129.0,0,0,NaN,NaN,0.0,0,33.495498,-112.066157
1,2,Southern Arizona VA Health Care System,VA Hospital,3601 S 6th Ave,NaN,Tucson,AZ,85723,Pima,Arizona,...,295.0,295.0,2,2,NaN,NaN,2.0,0,32.181263,-110.965885
2,3,VA Central California Health Care System,VA Hospital,2615 E Clinton Ave,NaN,Fresno,CA,93703,Fresno,California,...,57.0,57.0,2,2,NaN,NaN,2.0,0,36.773324,-119.779742
3,4,VA Connecticut Healthcare System - West Haven ...,VA Hospital,950 Campbell Ave,NaN,West Haven,CT,6516,New Haven,Connecticut,...,216.0,216.0,1,1,NaN,NaN,2.0,0,41.284400,-72.957610
4,5,Wilmington VA Medical Center,VA Hospital,1601 Kirkwood Hwy,NaN,Wilmington,DE,19805,New Castle,Delaware,...,60.0,60.0,0,0,NaN,NaN,1.0,0,39.740206,-75.606532


In [23]:
#Use Athena Client to run queries on rearc_usa_hospital_bedsrearc_usa_hospital_beds table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countrycodecountrycode",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '9c229a54-949a-4b3b-88ac-d670b3fa3edb',
 'ResponseMetadata': {'RequestId': '2748cd66-863d-402b-9312-52ccad72ba65',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:34 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2748cd66-863d-402b-9312-52ccad72ba65'},
  'RetryAttempts': 0}}

In [24]:
#call the function using athena_client, response of query as parameters
static_datasets_countrycodecountrycode=download_load_query(athena_client,response)

#static_datasets_countrycodecountrycode first 5 rows
static_datasets_countrycodecountrycode.head()

,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6


In [25]:
#Use Athena Client to run queries on rearc_usa_hospital_bedsrearc_usa_hospital_beds table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_countypopulationcountypopulation",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': 'a4b73a29-b3e8-4d82-bab5-2436346e95df',
 'ResponseMetadata': {'RequestId': '90ab8aa3-ee33-437c-8cdd-1863b8534cca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:36 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '90ab8aa3-ee33-437c-8cdd-1863b8534cca'},
  'RetryAttempts': 0}}

In [26]:
#call the function using athena_client, response of query as parameters
static_datasets_countypopulationcountypopulation=download_load_query(athena_client,response)

#static_datasets_countypopulationcountypopulation first 5 rows
static_datasets_countypopulationcountypopulation.head()

,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [27]:
#Use Athena Client to run queries on rearc_usa_hospital_bedsrearc_usa_hospital_beds table
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM static_datasets_state_abvstate_abv",
    QueryExecutionContext={"Database": SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
response

{'QueryExecutionId': '817dbc59-9c92-413e-9ac4-949648a2c73b',
 'ResponseMetadata': {'RequestId': '8c667e8a-d07c-4d80-a7dd-678bd5af79d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 02 May 2023 18:30:39 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '59',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8c667e8a-d07c-4d80-a7dd-678bd5af79d8'},
  'RetryAttempts': 0}}

In [28]:
#call the function using athena_client, response of query as parameters
static_datasets_state_abvstate_abv=download_load_query(athena_client,response)

#static_datasets_state_abvstate_abv first 5 rows
static_datasets_state_abvstate_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [29]:
#set first row as headers
header=static_datasets_state_abvstate_abv.iloc[0]
static_datasets_state_abvstate_abv=static_datasets_state_abvstate_abv[1:]
static_datasets_state_abvstate_abv.columns=header
static_datasets_state_abvstate_abv.head()

,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR
5,California,CA


# Transform current Relational data model to dimensional data model

In [30]:
#Extract the required columns from enigma_jhud
fact_Covid1=enigma_jhud[['fips','province_state','country_region','confirmed','deaths','recovered','active']]
#Extract the required columns from rearc_covid_19_testing_data_states_dailystates_daily
fact_Covid2=rearc_covid_19_testing_data_states_dailystates_daily[['fips','date','positive','negative','hospitalizedcurrently','hospitalized','hospitalizeddischarged']]

#merge columns from two tables to get fact covid table
fact_Covid=pd.merge(fact_Covid1,fact_Covid2,on='fips',how='inner')

In [31]:
fact_Covid.shape

(27992, 13)

In [32]:
#Extract the required columns from enigma_jhud
dim_Region1=enigma_jhud[['fips','province_state','country_region','latitude','longitude']]
#Extract the required columns from nytimes_data_in_usa_countyus_county
dim_Region2=nytimes_data_in_usa_countyus_county[['fips','county','state']]
#merge columns from two tables to get dim_Region
dim_Region=pd.merge(dim_Region1,dim_Region2,on='fips',how='inner')
dim_Region.shape

(11752274, 7)

In [33]:
#create dim_Hospital by extracting columns from rearc_usa_hospital_beds
dim_Hospital=rearc_usa_hospital_beds[['fips','state_name','latitude','longtitude','hq_address','hospital_name','hospital_type','hq_city','hq_state']]
dim_Hospital.shape

(829, 9)

In [34]:
#Create dim_Date by extracting existing columns from rearc_covid_19_testing_data_states_dailystates_daily 
dim_Date=rearc_covid_19_testing_data_states_dailystates_daily[['fips','date']]
#convert date column to required format
dim_Date['date']=pd.to_datetime(dim_Date['date'],format='%Y%m%d')
#Extract year,month,day_of_week from date column and add those columns to dim_Date
dim_Date['year']=dim_Date['date'].dt.year
dim_Date['month']=dim_Date['date'].dt.month
dim_Date['day_of_week']=dim_Date['date'].dt.dayofweek
dim_Date.shape

C:\Users\JAYAS\AppData\Local\Temp\ipykernel_2932\1994583441.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_Date['date']=pd.to_datetime(dim_Date['date'],format='%Y%m%d')
C:\Users\JAYAS\AppData\Local\Temp\ipykernel_2932\1994583441.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_Date['year']=dim_Date['date'].dt.year
C:\Users\JAYAS\AppData\Local\Temp\ipykernel_2932\1994583441.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

(2685, 5)

# Store the transformed data model to S3

In [35]:
bucket='cps-585-project'
csv_buffer=StringIO()
csv_buffer1=StringIO()
csv_buffer2=StringIO()
csv_buffer3=StringIO()

In [36]:
#convert the data from fact_Covid table to csv and store it in csv_buffer
fact_Covid.to_csv(csv_buffer)
#Connect to S3 and store the result in S3 bucket output folder
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/fact_Covid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'WA8CWCCJHKK6HPCE',
  'HostId': 'r4QzWLTnuUVkjzdJDDJYbShHzFV8UDpf6I2eCbxaE+s3eFuyQgydXiexWAEM/cbQ3shO6BtjSgM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'r4QzWLTnuUVkjzdJDDJYbShHzFV8UDpf6I2eCbxaE+s3eFuyQgydXiexWAEM/cbQ3shO6BtjSgM=',
   'x-amz-request-id': 'WA8CWCCJHKK6HPCE',
   'date': 'Tue, 02 May 2023 18:30:46 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"fc9be4f5fc50864df1f4ed534cf29988"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"fc9be4f5fc50864df1f4ed534cf29988"',
 'ServerSideEncryption': 'AES256'}

In [37]:
#convert the data from dim_Hospital table to csv and store it in csv_buffer
dim_Hospital.to_csv(csv_buffer1)
#Connect to S3 and store the result in S3 bucket output folder
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/dim_Hospital.csv').put(Body=csv_buffer1.getvalue())

{'ResponseMetadata': {'RequestId': '8XJB4NY90CGM78XH',
  'HostId': 'bG3ISF0/NCp3GnYd8ccVhbBc2LvLOU4Yb6le7j9z15kUb163ygoLn7giKBcP/xOIl1vC50Gl6I0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'bG3ISF0/NCp3GnYd8ccVhbBc2LvLOU4Yb6le7j9z15kUb163ygoLn7giKBcP/xOIl1vC50Gl6I0=',
   'x-amz-request-id': '8XJB4NY90CGM78XH',
   'date': 'Tue, 02 May 2023 18:30:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"e81caded3358841b6b8f6a6b8920270b"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"e81caded3358841b6b8f6a6b8920270b"',
 'ServerSideEncryption': 'AES256'}

In [38]:
#convert the data from dim_Date table to csv and store it in csv_buffer
dim_Date.to_csv(csv_buffer2)
#Connect to S3 and store the result in S3 bucket output folder
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/dim_Date.csv').put(Body=csv_buffer2.getvalue())

{'ResponseMetadata': {'RequestId': '8XJ481BXAX9KPQ73',
  'HostId': 'ACShK8AtqrHbAHvZT+9MHw8NYzJajsZ/OxL1PtnYs1A9yZW5C6TFar5a0xh8KutBa473mK352bA=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ACShK8AtqrHbAHvZT+9MHw8NYzJajsZ/OxL1PtnYs1A9yZW5C6TFar5a0xh8KutBa473mK352bA=',
   'x-amz-request-id': '8XJ481BXAX9KPQ73',
   'date': 'Tue, 02 May 2023 18:30:48 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"19eb0b77e7f7441c686829bc3fd1a906"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"19eb0b77e7f7441c686829bc3fd1a906"',
 'ServerSideEncryption': 'AES256'}

In [39]:
#convert the data from dim_Region table to csv and store it in csv_buffer
dim_Region.to_csv(csv_buffer3)
#Connect to S3 and store the result in S3 bucket output folder
s3_resource=boto3.resource('s3')
s3_resource.Object(bucket,'output/dim_Region.csv').put(Body=csv_buffer3.getvalue())

{'ResponseMetadata': {'RequestId': '7HAQR2NC9NBKHK21',
  'HostId': '9EaB3OpZmqDFlcu7PCyYUg5oGOHBdLH9Hrit5cVI8luK44JtXB38N6+vSiEHvZ5o/8JQOzLkIeU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9EaB3OpZmqDFlcu7PCyYUg5oGOHBdLH9Hrit5cVI8luK44JtXB38N6+vSiEHvZ5o/8JQOzLkIeU=',
   'x-amz-request-id': '7HAQR2NC9NBKHK21',
   'date': 'Tue, 02 May 2023 19:34:47 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"2fdf65265c31f0f6e8c02b4bd066d670"',
   'server': 'AmazonS3',
   'content-length': '0',
   'connection': 'close'},
  'RetryAttempts': 1},
 'ETag': '"2fdf65265c31f0f6e8c02b4bd066d670"',
 'ServerSideEncryption': 'AES256'}

In [40]:
#Extract schema out of dataframes
dim_Datesql=pd.io.sql.get_schema(dim_Date.reset_index(),'dim_Date')
print(''.join(dim_Datesql))
dim_Hospitalsql=pd.io.sql.get_schema(dim_Hospital.reset_index(),'dim_Hospital')
print(''.join(dim_Hospitalsql))
dim_Regionsql=pd.io.sql.get_schema(dim_Region.reset_index(),'dim_Region')
print(''.join(dim_Regionsql))
fact_Covidsql=pd.io.sql.get_schema(fact_Covid.reset_index(),'fact_Covid')
print(''.join(fact_Covidsql))

CREATE TABLE "dim_Date" (
"index" INTEGER,
  "fips" REAL,
  "date" TIMESTAMP,
  "year" INTEGER,
  "month" INTEGER,
  "day_of_week" INTEGER
)
CREATE TABLE "dim_Hospital" (
"index" INTEGER,
  "fips" INTEGER,
  "state_name" TEXT,
  "latitude" REAL,
  "longtitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
CREATE TABLE "dim_Region" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
CREATE TABLE "fact_Covid" (
"index" INTEGER,
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalizedcurrently" REAL,
  "hospitalized" REAL,
  "hospitalizeddischarged" REAL
)
